In [1]:
#%% Imports
from pathlib import Path
import numpy as np
import skimage.io
from tqdm import tqdm

# Import "local" packages
import hyspec_io, preprocess, image_render

In [2]:
# Parameters
nir_band = (730,800)

In [3]:
# Paths
image_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/2a_Rrs'),
              Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/Hyperspectral/20230620/2a_Rrs'),
              Path('/media/mha114/Massimal2/Massimal/Larvik_Olberg/Hyperspectral/20230830/2_Rrs'),
              Path('/media/mha114/Massimal2/Massimal/Larvik_Kongsbakkebukta/Hyperspectral/20230830/2_Rrs')]

In [4]:
for image_dir in image_dirs:
    print(f'Processing files in {image_dir}')
    # Create output dir
    image_output_dir = image_dir.parent / '3a_Rrs_FSSGC'
    image_output_dir.mkdir(exist_ok=True)
    
    # Find hyperspectral images
    image_paths = sorted(image_dir.glob('*.hdr'))
    
    for image_path in tqdm(image_paths):
        # Load image
        (image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(image_path)
        
        # Remove glint
        image_noglint = preprocess.remove_glint_flatspec(image,wl,nir_band=nir_band)

        # Save hyspec image
        image_save_path = image_output_dir / image_path.name
        hyspec_io.save_envi_image(image_save_path,image_noglint,metadata)

        # Create RGB version (percentile stretch)
        image_rgb = image_render.percentile_stretch(image_noglint[:,:,rgb_ind])
        image_rgb = np.uint8(image_rgb*255)  # Convert to 8-bit int
        transparency_layer = np.ones(shape=(image_rgb.shape[0:2])+(1,),dtype=np.uint8)*255
        image_rgb = np.concatenate((image_rgb,transparency_layer),axis=2)

        # Save RGB image
        hyspec_base_name = image_path.name.split('.')[0]
        rgg_image_save_path = image_output_dir / (hyspec_base_name + '.png')
        skimage.io.imsave(rgg_image_save_path,image_rgb,check_contrast=False)


Processing files in /media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/2a_Rrs


100%|██████████| 6/6 [01:54<00:00, 19.09s/it]


Processing files in /media/mha114/Massimal2/Massimal/Smola_Skalmen/Hyperspectral/20230620/2a_Rrs


100%|██████████| 8/8 [02:21<00:00, 17.72s/it]


Processing files in /media/mha114/Massimal2/Massimal/Larvik_Olberg/Hyperspectral/20230830/2_Rrs


100%|██████████| 3/3 [01:02<00:00, 20.94s/it]


Processing files in /media/mha114/Massimal2/Massimal/Larvik_Kongsbakkebukta/Hyperspectral/20230830/2_Rrs


  0%|          | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_1068658/2793951330.py:31: UserWarning: /media/mha114/Massimal2/Massimal/Larvik_Kongsbakkebukta/Hyperspectral/20230830/3a_Rrs_FSSGC/Kongsbakkebukta_Pika_L_1-Crop Wavelengths.png is a low contrast image
  skimage.io.imsave(rgg_image_save_path,image_rgb)
 50%|█████     | 1/2 [00:19<00:19, 19.19s/it]/tmp/ipykernel_1068658/2793951330.py:31: UserWarning: /media/mha114/Massimal2/Massimal/Larvik_Kongsbakkebukta/Hyperspectral/20230830/3a_Rrs_FSSGC/Kongsbakkebukta_Pika_L_4-Crop Wavelengths.png is a low contrast image
  skimage.io.imsave(rgg_image_save_path,image_rgb)
100%|██████████| 2/2 [00:34<00:00, 17.03s/it]
